# Context Extractors
> Extractor interfaces and implementations for extracting context from function calls


In [ ]:
#| default_exp client.extractors

In [ ]:
#| export
from abc import ABC, abstractmethod
from typing import Optional, Callable, Any, Dict, Tuple

from dc_logger.client.Log import LogEntity, HTTPDetails, MultiTenant

## Extractor Interfaces

These abstract base classes define the contract for extracting different types of context information from function calls.

Following the **Interface Segregation Principle**, each extractor has a single, focused responsibility.


In [ ]:
#| export

class EntityExtractor(ABC):
    """Abstract base class for extracting entity information from function arguments."""
    
    @abstractmethod
    def extract(self, func: Callable, args: tuple, kwargs: dict) -> Optional[LogEntity]:
        """Extract entity information from function call.
        
        Args:
            func: The function being called
            args: Positional arguments
            kwargs: Keyword arguments
            
        Returns:
            LogEntity or None if no entity found
        """
        pass


In [ ]:
#| export

class HTTPDetailsExtractor(ABC):
    """Abstract base class for extracting HTTP details from function arguments."""
    
    @abstractmethod
    def extract(self, func: Callable, args: tuple, kwargs: dict) -> Optional[HTTPDetails]:
        """Extract HTTP details from function call.
        
        Args:
            func: The function being called
            args: Positional arguments
            kwargs: Keyword arguments
            
        Returns:
            HTTPDetails or None if no HTTP details found
        """
        pass


In [ ]:
#| export

class MultiTenantExtractor(ABC):
    """Abstract base class for extracting multi-tenant information from function arguments."""
    
    @abstractmethod
    def extract(self, func: Callable, args: tuple, kwargs: dict) -> Optional[MultiTenant]:
        """Extract multi-tenant information from function call.
        
        Args:
            func: The function being called
            args: Positional arguments
            kwargs: Keyword arguments
            
        Returns:
            MultiTenant or None if no multi-tenant info found
        """
        pass


In [ ]:
#| export

class ResultProcessor(ABC):
    """Abstract base class for processing function results."""
    
    @abstractmethod
    def process(self, result: Any, http_details: Optional[HTTPDetails] = None) -> Tuple[Dict[str, Any], Optional[HTTPDetails]]:
        """Process function result and optionally update HTTP details.
        
        Args:
            result: The function result
            http_details: Optional HTTP details to update
            
        Returns:
            Tuple of (result_context dict, updated http_details)
        """
        pass


## Default Implementations

These are sensible default implementations that work with keyword arguments.
You can extend or replace these for domain-specific behavior.


In [ ]:
#| export

class KwargsEntityExtractor(EntityExtractor):
    """Default entity extractor that looks for entity in kwargs."""
    
    def __init__(self, kwarg_name: str = "entity"):
        self.kwarg_name = kwarg_name
    
    def extract(self, func: Callable, args: tuple, kwargs: dict) -> Optional[LogEntity]:
        entity = kwargs.get(self.kwarg_name)
        return LogEntity.from_any(entity) if entity else None


In [ ]:
#| export

class KwargsHTTPDetailsExtractor(HTTPDetailsExtractor):
    """Default HTTP details extractor that looks for common HTTP kwargs."""
    
    def extract(self, func: Callable, args: tuple, kwargs: dict) -> Optional[HTTPDetails]:
        # Check if http_details is directly provided
        if "http_details" in kwargs:
            hd = kwargs["http_details"]
            if isinstance(hd, HTTPDetails):
                return hd
            elif isinstance(hd, dict):
                return HTTPDetails(**hd)
        
        # Otherwise, try to construct from common kwargs
        if any(k in kwargs for k in ["method", "url", "headers"]):
            return HTTPDetails(
                method=kwargs.get("method"),
                url=kwargs.get("url"),
                headers=kwargs.get("headers"),
                params=kwargs.get("params"),
                request_body=kwargs.get("body") or kwargs.get("request_body"),
            )
        
        return None


In [ ]:
#| export

class KwargsMultiTenantExtractor(MultiTenantExtractor):
    """Default multi-tenant extractor that looks for multi_tenant in kwargs."""
    
    def extract(self, func: Callable, args: tuple, kwargs: dict) -> Optional[MultiTenant]:
        return MultiTenant.from_kwargs(kwargs)


In [ ]:
#| export

class DefaultResultProcessor(ResultProcessor):
    """Default result processor with configurable result inclusion."""
    
    def __init__(self, include_result: bool = False, max_result_length: int = 100):
        self.include_result = include_result
        self.max_result_length = max_result_length
    
    def process(self, result: Any, http_details: Optional[HTTPDetails] = None) -> Tuple[Dict[str, Any], Optional[HTTPDetails]]:
        result_context = {}
        
        # Update HTTP details if result has status/response attributes
        if http_details and hasattr(result, "status"):
            http_details.status_code = getattr(result, "status", None)
            
            if hasattr(result, "response"):
                response = result.response
                if isinstance(response, (str, bytes)):
                    http_details.response_size = len(response)
                    http_details.response_body = str(response)[:500] if len(str(response)) > 500 else str(response)
                elif hasattr(response, "__len__"):
                    try:
                        http_details.response_size = len(response)
                    except:
                        pass
        
        # Add result to context if requested
        if self.include_result and result is not None:
            if hasattr(result, "__len__") and len(result) > self.max_result_length:
                result_context["result"] = f"<{type(result).__name__} with {len(result)} items>"
            elif isinstance(result, (str, int, float, bool, type(None))):
                result_context["result"] = result
            else:
                result_context["result"] = f"<{type(result).__name__}>"
        
        return result_context, http_details


## Example: Custom Extractor

Here's an example of how to create a custom extractor for domain-specific needs.


In [ ]:
# Example: Custom extractor for specific auth objects
# This example shows how you might extract entity info from an auth object

# class AuthBasedEntityExtractor(EntityExtractor):
#     """Extract entity from authentication object."""
#     
#     def extract(self, func: Callable, args: tuple, kwargs: dict) -> Optional[LogEntity]:
#         auth = kwargs.get("auth")
#         
#         # Extract from auth if it has specific attributes
#         if auth and hasattr(auth, "instance_id"):
#             return LogEntity(
#                 type="instance",
#                 id=getattr(auth, "instance_id", None),
#                 name=getattr(auth, "instance_name", None)
#             )
#         
#         # Fallback to checking kwargs for entity
#         entity = kwargs.get("entity")
#         return LogEntity.from_any(entity) if entity else None


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export('./extractors.ipynb')
